In [1]:
import os
import sys
import json
import io
import hashlib

import numpy as np

import pandas as pd

from PIL import Image

REPO_DIR = os.path.join(os.environ['HOME'], 'github_repos')
sys.path.append(os.path.join(REPO_DIR, 'partial-data/'))
from partial_data.visualization import draw_bounding_boxes_on_image
from partial_data.tfrecord import (create_label_map_pbtxt, encode_object_detection_tf_example, 
                                   decode_object_detection_tf_example, write_examples_as_tfrecord, 
                                   read_examples_from_tfrecord)

## Load and process data 

In [2]:
coco_image_dir = '/home/david/datasets/coco/val2017/images/'
coco_annotations_filepath = '/home/david/datasets/coco/val2017/annotations/instances_val2017.json'

In [3]:
# Load raw data from disk
with open(coco_annotations_filepath, 'r') as fp:
    val_instances = json.load(fp)

In [4]:
# Load annotation info into dataframe
df_ann = (
    pd.DataFrame(val_instances['annotations'])
    .drop(columns=['segmentation', 'area'])
    .rename(columns={'id': 'instance_id'})
)

# Load category info into dataframe
df_cat = (
    pd.DataFrame(val_instances['categories'])
    .drop(columns=['supercategory'])
    .rename(columns={'id': 'category_id', 'name': 'category_name'})
)
df_cat['category_name_orig'] = df_cat['category_name']
df_cat['category_name'] = df_cat['category_name'].str.lower().str.replace(' ', '_')

# Load image info
df_image = pd.DataFrame(val_instances['images']).rename(columns={'id': 'image_id'})
df_image['image_filepath'] = df_image['file_name'].apply(lambda x: os.path.join(coco_image_dir, x))
df_image.drop(columns=['license', 'file_name', 'coco_url', 'date_captured', 'flickr_url'], inplace=True)

# Combine all info into single dataframe
df_comb = pd.merge(df_ann, df_cat, on='category_id', how='left')
df_comb = pd.merge(df_comb, df_image, on='image_id', how='left')

In [5]:
# Convert bounding boxes to normalized coordinates
# Original box coordinates are [x,y,width,height], measured from the top left image corner, and 0-indexed
df_comb['wmin'] = df_comb['bbox'].str[0] / df_comb['width']
df_comb['hmin'] = df_comb['bbox'].str[1] / df_comb['height']
df_comb['wmax'] = (df_comb['bbox'].str[0] + df_comb['bbox'].str[2]) / df_comb['width']
df_comb['hmax'] = (df_comb['bbox'].str[1] + df_comb['bbox'].str[3]) / df_comb['height']

## Visualize bounding boxes 

In [7]:
ind = 0

image_grps = df_comb.groupby('image_filepath')
image_filepaths = list(image_grps.groups.keys())
image_filepath = image_filepaths[ind]
df_image_grp = image_grps.get_group(image_filepath)
bboxes = df_image_grp[['hmin', 'wmin', 'hmax', 'wmax']].values
labels = [(nm,) for nm in df_image_grp['category_name']]
img = Image.open(image_filepath)
font_filepath = '/home/david/github_repos/fonts/DejaVuSansMono/DejaVu Sans Mono for Powerline.ttf'
draw_bounding_boxes_on_image(img, bboxes, 
                             font_size=18, font_filepath=font_filepath, 
                             display_str_list_list=labels)
img

## Create datasets 

In [8]:
# Choose categories to consider, implicitly indicating which should have missing instances
dataset1_cats = {'person', 'car', 'book', 'cup', 'dining_table'}
dataset2_cats = {'person', 'chair', 'bottle', 'cup', 'dining_table'}
dataset3_cats = dataset1_cats | dataset2_cats

# Assign images to differnet halves of dataset with missing annotations
np.random.seed(0)
image_ids = df_comb['image_id'].unique()
image_id_dataset_mask = np.random.randint(0, 2, len(image_ids)).astype(bool)
image_ids_dataset1 = set(image_ids[image_id_dataset_mask].tolist())
image_ids_dataset2 = set(image_ids[~image_id_dataset_mask].tolist())
is_dataset1_image = df_comb['image_id'].isin(image_ids_dataset1)
is_dataset2_image = df_comb['image_id'].isin(image_ids_dataset2)

# Identify instances belonging to valid categories
is_dataset1_cat = df_comb['category_name'].isin(dataset1_cats)
is_dataset2_cat = df_comb['category_name'].isin(dataset2_cats)
is_dataset3_cat = df_comb['category_name'].isin(dataset3_cats)

# Identify valid instances for two halves of dataset with partial annotations
is_dataset1_instance = is_dataset1_image & is_dataset1_cat
is_dataset2_instance = is_dataset2_image & is_dataset2_cat
is_dataset12_instance = is_dataset1_instance | is_dataset2_instance

# Identify valid instances for dataset with complete annotations
image_ids_dataset12 = set(df_comb.loc[is_dataset12_instance, 'image_id'].tolist())
is_dataset3_image = df_comb['image_id'].isin(image_ids_dataset12)
is_dataset3_instance = is_dataset3_image & is_dataset3_cat

# Create dataframes for datasets with partial and complete annotations
df_dataset12 = df_comb[is_dataset12_instance].copy()
df_dataset3 = df_comb[is_dataset3_instance].copy()

# Create new sequential label ids for retained categories
category_id_map_new = {nm: ind for ind, nm in enumerate(dataset3_cats, 1)}
df_dataset12['category_id_orig'] = df_dataset12['category_id']
df_dataset3['category_id_orig'] = df_dataset3['category_id']
df_dataset12['category_id'] = df_dataset12['category_name'].apply(lambda x: category_id_map_new[x])
df_dataset3['category_id'] = df_dataset3['category_name'].apply(lambda x: category_id_map_new[x])

In [9]:
# Get total instance and image counts
num_images1 = df_comb.loc[is_dataset1_instance, 'image_id'].nunique()
num_images2 = df_comb.loc[is_dataset2_instance, 'image_id'].nunique()
num_images3 = df_comb.loc[is_dataset3_instance, 'image_id'].nunique()
print(f'dataset 1 - num images: {num_images1}, num instances: {is_dataset1_instance.sum()}')
print(f'dataset 2 - num images: {num_images2}, num instances: {is_dataset2_instance.sum()}')
print(f'dataset 3 - num images: {num_images3}, num instances: {is_dataset3_instance.sum()}')

# Get per category instance counts 
pd.concat(
    [
        df_comb.loc[is_dataset1_instance, 'category_name'].value_counts().rename('dataset1'),
        df_comb.loc[is_dataset2_instance, 'category_name'].value_counts().rename('dataset2'),
        df_comb.loc[is_dataset3_instance, 'category_name'].value_counts().rename('dataset3'),
    ],
    axis=1
).sort_values(by='dataset3', ascending=False).astype(pd.Int64Dtype())

dataset 1 - num images: 1681, num instances: 7986
dataset 2 - num images: 1590, num instances: 7503
dataset 3 - num images: 3271, num instances: 17919


dataset1  dataset2  dataset3
person            5630      5374     11004
chair             <NA>       802      1712
car                915      <NA>      1650
book               583      <NA>      1020
bottle            <NA>       589       937
cup                465       434       899
dining_table       393       304       697

## Split datasets into train/val/test

In [10]:
# Define split parameters
train_frac = 0.7
val_frac = 0.1
np.random.seed(0)

# Assign images to different splits
num_train = round(len(image_ids) * train_frac)
num_val = round(len(image_ids) * val_frac)
image_ids_shuffle = np.random.permutation(image_ids)
image_ids_train = set(image_ids_shuffle[:num_train].tolist())
image_ids_val = set(image_ids_shuffle[num_train:num_train+num_val].tolist())
image_ids_test = set(image_ids_shuffle[num_train+num_val:].tolist())

# Create dataframes with different splits
df_dataset12_train = df_dataset12[df_dataset12['image_id'].isin(image_ids_train)]
df_dataset12_val = df_dataset12[df_dataset12['image_id'].isin(image_ids_val)]
df_dataset12_test = df_dataset12[df_dataset12['image_id'].isin(image_ids_test)]
df_dataset3_train = df_dataset3[df_dataset3['image_id'].isin(image_ids_train)]
df_dataset3_val = df_dataset3[df_dataset3['image_id'].isin(image_ids_val)]
df_dataset3_test = df_dataset3[df_dataset3['image_id'].isin(image_ids_test)]

In [11]:
# Check image and instance counts per split
df_split_count = pd.DataFrame(
    [
        [df_dataset12_train['image_id'].nunique(), df_dataset12_val['image_id'].nunique(), df_dataset12_test['image_id'].nunique()],
        [df_dataset3_train['image_id'].nunique(), df_dataset3_val['image_id'].nunique(), df_dataset3_test['image_id'].nunique()],
        [len(df_dataset12_train), len(df_dataset12_val), len(df_dataset12_test)],
        [len(df_dataset3_train), len(df_dataset3_val), len(df_dataset3_test)]
    ],
    index=['num_images_12', 'num_images_3', 'num_instances_12', 'num_instances_3'],
    columns=['train', 'val', 'test']
)
df_split_count['all'] = df_split_count.sum(axis=1)
df_split_count

train   val  test    all
num_images_12      2315   321   635   3271
num_images_3       2315   321   635   3271
num_instances_12  10952  1539  2998  15489
num_instances_3   12616  1768  3535  17919

## Save annotation datasets to disk 

In [12]:
output_dir = './annotations'
dataset12_prefix = 'annotations_partial_'
dataset3_prefix = 'annotations_complete_'

df_dataset12_train.to_csv(os.path.join(output_dir, dataset12_prefix+'train.csv'))
df_dataset12_val.to_csv(os.path.join(output_dir, dataset12_prefix+'val.csv'))
df_dataset12_test.to_csv(os.path.join(output_dir, dataset12_prefix+'test.csv'))
df_dataset3_train.to_csv(os.path.join(output_dir, dataset3_prefix+'train.csv'))
df_dataset3_val.to_csv(os.path.join(output_dir, dataset3_prefix+'val.csv'))
df_dataset3_test.to_csv(os.path.join(output_dir, dataset3_prefix+'test.csv'))

## Create .pbtxt file 

In [13]:
label_map = [{'name': nm, 'display_name': nm, 'id': ind,} for nm, ind in category_id_map_new.items()]
create_label_map_pbtxt(label_map, 'tfrecord/label_map.pbtxt')

## Create TFRecords 

In [14]:
agg_cols = ['wmin', 'wmax', 'hmin', 'hmax', 'category_name', 'category_id']
grp_cols = ['image_filepath', 'image_id']

dataset_dfs = [df_dataset12_train, df_dataset12_val, df_dataset12_test, 
               df_dataset3_train, df_dataset3_val, df_dataset3_test]
tfrecord_output_filepaths = [f'tfrecord/partial_{split}.record' for split in ['train', 'val', 'test']] + \
                            [f'tfrecord/complete_{split}.record' for split in ['train', 'val', 'test']]

for output_filepath, df_dataset in zip(tfrecord_output_filepaths, dataset_dfs):
    print(output_filepath)
    df_example = df_dataset.groupby(grp_cols).agg({col: list for col in agg_cols}).reset_index()
    examples = [row for _, row in df_example.iterrows()]
    write_examples_as_tfrecord(
        examples,
        output_filepath,
        encode_object_detection_tf_example,
        num_shards=3
    )

tfrecord/partial_train.record


100%|██████████| 2315/2315 [00:04<00:00, 473.29it/s]


tfrecord/partial_val.record


100%|██████████| 321/321 [00:00<00:00, 375.20it/s]


tfrecord/partial_test.record


...

100%|██████████| 2315/2315 [00:06<00:00, 373.38it/s]


tfrecord/complete_val.record


100%|██████████| 321/321 [00:00<00:00, 422.17it/s]


tfrecord/complete_test.record


100%|██████████| 635/635 [00:01<00:00, 523.92it/s]
